# **Assignment #2**
###	Bank Marketing Campaign Data Analysis with DataFrame API 
1.   Load Bank Marketing Dataset and create DataFrame
2.   Give marketing success rate. (No. of people subscribed / total no. of entries)
3.   Give marketing failure rate
4.   Maximum, Mean, and Minimum age of the average targeted customer
5.   Check the quality of customers by checking the average balance, median balance of customers
6.   Check if age matters in marketing subscription for deposit
7.   Show AgeGroup [Teenagers, Youngsters, MiddleAgers, Seniors] wise Subscription Count.
8.   Check if marital status mattered for subscription to deposit.
9.   Check if age and marital status together mattered for subscription to deposit scheme

In [1]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=3eda868aaf357005bcd9f1f0187e4a4c44c804928dee50b408e1bbfea4c0ee47
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('pyspark-examples').getOrCreate()

In [15]:
df = spark.read.options(delimiter=';',header=True,inferSchema=True).csv("bankmarketdata.csv")

In [16]:
df.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [18]:
df.registerTempTable("bankmarketing")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [19]:
spark.sql("select (sum(if(y='yes',1,0))/count(*))*100 from bankmarketing").show()

+-----------------------------------------------+
|((sum((IF((y = yes), 1, 0))) / count(1)) * 100)|
+-----------------------------------------------+
|                             11.698480458295547|
+-----------------------------------------------+



In [20]:
spark.sql("select max(age),min(age),avg(age) from bankmarketing").show()

+--------+--------+-----------------+
|max(age)|min(age)|         avg(age)|
+--------+--------+-----------------+
|      95|      18|40.93621021432837|
+--------+--------+-----------------+



In [21]:
spark.sql("select avg(balance),percentile_approx(balance,0.5) as medium_balance from bankmarketing").show()

+------------------+--------------+
|      avg(balance)|medium_balance|
+------------------+--------------+
|1362.2720576850766|           448|
+------------------+--------------+



In [22]:
spark.sql('select age,count(*) cnt from bankmarketing where y="yes" group by age order by cnt desc ').show()

+---+---+
|age|cnt|
+---+---+
| 32|221|
| 30|217|
| 33|210|
| 35|209|
| 31|206|
| 34|198|
| 36|195|
| 29|171|
| 37|170|
| 28|162|
| 38|144|
| 39|143|
| 27|141|
| 26|134|
| 41|120|
| 46|118|
| 40|116|
| 47|113|
| 25|113|
| 42|111|
+---+---+
only showing top 20 rows



In [23]:
spark.sql("select (case when age<13 then 'Kids' when age<20 then 'Teenagers' when age<31 then 'Youngsters' when age<50 then 'Middleagers 'else 'Seniors' end) as ag,count(*) sub_cnt from bankmarketing where y='yes' group by ag order by sub_cnt desc ").show()

+------------+-------+
|          ag|sub_cnt|
+------------+-------+
|Middleagers |   2759|
|     Seniors|   1385|
|  Youngsters|   1127|
|   Teenagers|     18|
+------------+-------+



In [26]:
spark.sql("select marital, count(*) as number from bankmarketing where y='yes' group by marital order by number desc").show()

+--------+------+
| marital|number|
+--------+------+
| married|  2755|
|  single|  1912|
|divorced|   622|
+--------+------+



In [27]:
spark.sql("select age, marital, count(*) as number from bankmarketing where y='yes' group by age,marital order by number desc").show()

+---+-------+------+
|age|marital|number|
+---+-------+------+
| 30| single|   151|
| 28| single|   138|
| 29| single|   133|
| 32| single|   124|
| 26| single|   121|
| 34|married|   118|
| 31| single|   111|
| 27| single|   110|
| 35|married|   101|
| 36|married|   100|
| 25| single|    99|
| 37|married|    98|
| 33| single|    97|
| 33|married|    97|
| 32|married|    87|
| 39|married|    87|
| 38|married|    86|
| 35| single|    84|
| 47|married|    83|
| 46|married|    80|
+---+-------+------+
only showing top 20 rows

